# SITA & Differential Privacy on CO2 prediction 

### 1. Instalação das dependências externas



In [3]:
pip install diffprivlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.9/176.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 82.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 87.2 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
catboost 1.2.7 requires numpy<2.0,>=1.16.0, but you have numpy 2.2.6 which is incompatible.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but yo

### 2. Importação as dependências



In [7]:
import pandas as pd
import numpy as np
import sklearn
import statistics
import warnings

warnings.filterwarnings("ignore")

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from diffprivlib.mechanisms import Laplace


### 3. Definição da configuração SITA

- Mapeamento das colunas do dataset inicial para as respectivas dimensões do SITA.
- Valores do parâmetro de privacidade(_epsilon_) para os níveis 0 e 3 modelo SITA.


In [10]:
epsilon_level_0 = 0.2
epsilon_level_1 = 0.4
epsilon_level_2 = 0.6
epsilon_level_3 = 0.8

spatial_config = {
    3: {
        "columns": ['Room','Zone'],
        "laplaceConfig": {
            "epsilon": epsilon_level_3
        }
    },
    2: {
        "columns": ['Room', 'Zone'],
        "laplaceConfig": {
            "epsilon": epsilon_level_2
        }
    }, 
    1: {
        "columns": ['Room', 'Zone'],
        "laplaceConfig": {
            "epsilon": epsilon_level_1
        }
    },
    0: {
        "columns": ['Room', 'Zone'],
        "laplaceConfig": {
            "epsilon": epsilon_level_0
        }
    }
}

temporal_config = {
    3: {
        "columns": ['Date', 'Time'],
        "laplaceConfig": {
            "epsilon": epsilon_level_3
        }
    },
    2: {
        "columns": ['Date', 'Time'],
        "laplaceConfig": {
            "epsilon": epsilon_level_2
        }
    }, 
    1: {
        "columns": ['Date', 'Time'],
        "laplaceConfig": {
            "epsilon": epsilon_level_1
        }
    },
    0: {
        "columns": ['Date', 'Time'],
        "laplaceConfig": {
            "epsilon": epsilon_level_0
        }
    }
}

activity_config = {
    3: {
        "columns": ['CO2', 'Humidity', 'Temperature', 'Brightness'],
        "laplaceConfig": {
            "epsilon": epsilon_level_3
        }
    },
    2: {
        "columns": ['CO2', 'Humidity', 'Temperature', 'Brightness'],
        "laplaceConfig": {
            "epsilon": epsilon_level_2
        }
    }, 
    1: {
        "columns": ['CO2', 'Humidity', 'Temperature', 'Brightness'],
        "laplaceConfig": {
            "epsilon": epsilon_level_1
        }
    },
    0: {
        "columns": ['CO2', 'Humidity', 'Temperature', 'Brightness'],
        "laplaceConfig": {
            "epsilon": epsilon_level_0
        }
    }
}

sitaConfig = {
    "spatial": spatial_config,
    "temporal": temporal_config,
    "activity": activity_config
}

### 4. Função que aplica o modelo conceitual SITA usando como estratégia de privacidade o ruído de Laplace

A definição da configuração é realizado na célula acima e disponível na variável `sitaConfig`.

In [12]:
def applySITAandLaplaceNoise(dataframe, dimension, level):
    
    dimension_level_config = sitaConfig[dimension][level]
    columns =  dimension_level_config["columns"]

    # DP mechanism parameters
    epsilon = dimension_level_config["laplaceConfig"]["epsilon"]
    delta = 0.0
    sensitivity = 1.0
    random_state=10

    noiser = Laplace(epsilon=epsilon, delta=delta, sensitivity=sensitivity, random_state=random_state)

    copy_df = dataframe.copy()
    
    for index, row in dataframe.iterrows():
        for column in columns: 
           copy_df.loc[index, column] = noiser.randomise(row[column])

    return copy_df

### 5. Função que lê e trata os dados do _dataset_ inicial 

Os dados presentes no dataset _inicial_ foram coletados pelos sensores presentes no USB de Newcastle. A extração e coleta dos dados foi realizado pela equipe da CONSEG da PUC RS.

As seguintes colunas estão presentes no _dataset_:
- Room
- Zone
- Date
- Time
- CO2
- Temperature
- Humidity
- Brightness
- Occupancy

In [13]:
def readOriginalCSV():
    file_src = '/kaggle/input/urban-observatory/historical_data_occupancy_4444.csv'

    data = pd.read_csv(file_src, sep=",", index_col=False)

    data['Room'] = pd.to_numeric(data['Room'], errors='coerce')
    data['Zone'] = pd.to_numeric(data['Zone'], errors='coerce')
    data['Date'] = pd.to_numeric(data['Date'], errors='coerce')
    data['Time'] = pd.to_numeric(data['Time'], errors='coerce')
    data['CO2'] = pd.to_numeric(data['CO2'], errors='coerce')
    data['Temperature'] = pd.to_numeric(data['Temperature'], errors='coerce')
    data['Humidity'] = pd.to_numeric(data['Humidity'], errors='coerce')
    data['Brightness'] = pd.to_numeric(data['Brightness'], errors='coerce')
    data['Occupancy'] = pd.to_numeric(data['Occupancy'], errors='coerce')
    
    return data

### 6. Aplicação da função `applySITAandLaplaceNoise` para geração dos novos _datasets_

A função `applySITAandLaplaceNoise` é aplicada sobre o _dataset_ inicial para cada uma das dimensões e niveis do modelo SITA, fazendo assim que sejam gerado 12 _datasets_ privados.  

In [14]:
dataframe = readOriginalCSV()

dataset_3444  = applySITAandLaplaceNoise(dataframe, "spatial", 3)
dataset_3444.to_csv('/kaggle/working/historical_data_occupancy_with_dp_3444.csv', sep=',', index=False)

dataset_2444  = applySITAandLaplaceNoise(dataframe, "spatial", 2)
dataset_2444.to_csv('/kaggle/working/historical_data_occupancy_with_dp_2444.csv', sep=',', index=False)

dataset_1444  = applySITAandLaplaceNoise(dataframe, "spatial", 1)
dataset_1444.to_csv('/kaggle/working/historical_data_occupancy_with_dp_1444.csv', sep=',', index=False)

dataset_0444  = applySITAandLaplaceNoise(dataframe, "spatial", 0)
dataset_0444.to_csv('/kaggle/working/historical_data_occupancy_with_dp_0444.csv', sep=',', index=False)

dataset_4434  = applySITAandLaplaceNoise(dataframe, "temporal", 3)
dataset_4434.to_csv('/kaggle/working/historical_data_occupancy_with_dp_4434.csv', sep=',', index=False)

dataset_4424  = applySITAandLaplaceNoise(dataframe, "temporal", 2)
dataset_4424.to_csv('/kaggle/working/historical_data_occupancy_with_dp_4424.csv', sep=',', index=False)

dataset_4414  = applySITAandLaplaceNoise(dataframe, "temporal", 1)
dataset_4414.to_csv('/kaggle/working/historical_data_occupancy_with_dp_4414.csv', sep=',', index=False)

dataset_4404  = applySITAandLaplaceNoise(dataframe, "temporal", 0)
dataset_4404.to_csv('/kaggle/working/historical_data_occupancy_with_dp_4404.csv', sep=',', index=False)

dataset_4443  = applySITAandLaplaceNoise(dataframe, "activity", 3)
dataset_4443.to_csv('/kaggle/working/historical_data_occupancy_with_dp_4443.csv', sep=',', index=False)

dataset_4442  = applySITAandLaplaceNoise(dataframe, "activity", 2)
dataset_4442.to_csv('/kaggle/working/historical_data_occupancy_with_dp_4442.csv', sep=',', index=False)

dataset_4441  = applySITAandLaplaceNoise(dataframe, "activity", 1)
dataset_4441.to_csv('/kaggle/working/historical_data_occupancy_with_dp_4441.csv', sep=',', index=False)

dataset_4440  = applySITAandLaplaceNoise(dataframe, "activity", 0)
dataset_4440.to_csv('/kaggle/working/historical_data_occupancy_with_dp_4440.csv', sep=',', index=False)


### 7. Treinamento e avaliação do modelo de _machine_ _learning_ _Random_ _Forest_ sobre o _dataset_ inicial e os _datasets_ privados

A predição de CO2 é feita através do algoritmo _Random_ _Forest_.

- Cada _dataset_ é segmentado em 80% para treinamento e 20% para testes.
- É usado a classe _RandomForestRegressor_ para o treinamento e predição do modelo.
- A avaliação do modelo é feita através do _cross_val_score_ e as métricas R2, MAE, e RMSE são extraídas.


In [20]:
files = [
    '/kaggle/input/urban-observatory/historical_data_occupancy_4444.csv',
    '/kaggle/working/historical_data_occupancy_with_dp_3444.csv',
    '/kaggle/working/historical_data_occupancy_with_dp_2444.csv',
    '/kaggle/working/historical_data_occupancy_with_dp_1444.csv',
    '/kaggle/working/historical_data_occupancy_with_dp_0444.csv',
    '/kaggle/working/historical_data_occupancy_with_dp_4434.csv',
    '/kaggle/working/historical_data_occupancy_with_dp_4424.csv',
    '/kaggle/working/historical_data_occupancy_with_dp_4414.csv',
    '/kaggle/working/historical_data_occupancy_with_dp_4404.csv',
    '/kaggle/working/historical_data_occupancy_with_dp_4443.csv',
    '/kaggle/working/historical_data_occupancy_with_dp_4442.csv',
    '/kaggle/working/historical_data_occupancy_with_dp_4441.csv',
    '/kaggle/working/historical_data_occupancy_with_dp_4440.csv'
]

In [23]:
def resultsPredictionKFold(X_train, y_train, cv, model):
    mae = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv)
    rmse = cross_val_score(model, X_train, y_train, scoring='neg_root_mean_squared_error', cv=cv)
    r2 = cross_val_score(model, X_train, y_train, scoring='r2', cv=cv)
    result = [r2, mae, rmse]
    return result

In [ ]:
np.random.seed(10)

config_sita = []
modelo_ml = []
r2 = []
mae = []
rmse = []

for filename in files:    
    print()
    print(filename)
    print()
    
    sita = filename.split('.')[0][-4:]
    
    data = pd.read_csv(filename, sep=",", index_col=False)
    
    data['Room'] = pd.to_numeric(data['Room'], errors='coerce')
    data['Zone'] = pd.to_numeric(data['Zone'], errors='coerce')
    data['Date'] = pd.to_numeric(data['Date'], errors='coerce')
    data['Time'] = pd.to_numeric(data['Time'], errors='coerce')
    data['CO2'] = pd.to_numeric(data['CO2'], errors='coerce')
    data['Temperature'] = pd.to_numeric(data['Temperature'], errors='coerce')
    data['Humidity'] = pd.to_numeric(data['Humidity'], errors='coerce')
    data['Brightness'] = pd.to_numeric(data['Brightness'], errors='coerce')
    data['Occupancy'] = pd.to_numeric(data['Occupancy'], errors='coerce')

    X = data[['Room', 'Zone', 'Date', 'Time', 'Temperature', 'Humidity', 'Brightness', 'Occupancy']]
    y = data['CO2']
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
    cv = KFold(n_splits=10, shuffle=True)
    
    modelRF = RandomForestRegressor(random_state=10, n_jobs=-1)
    modelRF.fit(X_train, y_train) 
    y_pred = modelRF.predict(X_test)
    resultsRF_KFold = resultsPredictionKFold(X_train, y_train, cv, modelRF)
    
    print("Modelo RF")
    print()
    config_sita.append(sita)
    modelo_ml.append("RF")
    

    print("Métrica R2")
    print("Média: %.6f" %(abs(resultsRF_KFold[0].mean())))
    r2.append(abs(resultsRF_KFold[0].mean()))
    print("Desvio Padrão: %.6f" %abs(resultsRF_KFold[0].std()))
    print("Mediana: %.6f" %abs(statistics.median(resultsRF_KFold[0])))
    print()

    print("Métrica MAE")
    print("Média: %.6f" %abs(resultsRF_KFold[1].mean()))
    mae.append(abs(resultsRF_KFold[1].mean()))
    print("Desvio Padrão: %.6f" %abs(resultsRF_KFold[1].std()))
    print("Mediana: %.6f" %abs(statistics.median(resultsRF_KFold[1])))
    print()

    print("Métrica RMSE")
    print("Média: %.6f" %abs(resultsRF_KFold[2].mean()))
    rmse.append(abs(resultsRF_KFold[2].mean()))
    print("Desvio Padrão: %.6f" %abs(resultsRF_KFold[2].std()))
    print("Mediana: %.6f" %abs(statistics.median(resultsRF_KFold[2])))
    print()
        
result = {
    "config_sita": config_sita, 
    "modelo_ml": modelo_ml,
    "r2": r2,
    "mae": mae,
    "rmse": rmse
}

result_df = pd.DataFrame(result)
result_df.to_csv('/kaggle/working/result.csv', sep=',', index=False)


/kaggle/input/urban-observatory/historical_data_occupancy_4444.csv

Modelo RF

Métrica R2
Média: 0.731265
Desvio Padrão: 0.006549
Mediana: 0.729779

Métrica MAE
Média: 32.311984
Desvio Padrão: 0.221125
Mediana: 32.257083

Métrica RMSE
Média: 62.865540
Desvio Padrão: 0.980537
Mediana: 62.758568


/kaggle/working/historical_data_occupancy_with_dp_3444.csv

Modelo RF

Métrica R2
Média: 0.684340
Desvio Padrão: 0.006658
Mediana: 0.685174

Métrica MAE
Média: 40.781522
Desvio Padrão: 0.314230
Mediana: 40.812021

Métrica RMSE
Média: 68.251836
Desvio Padrão: 0.476462
Mediana: 68.273168


/kaggle/working/historical_data_occupancy_with_dp_2444.csv

Modelo RF

Métrica R2
Média: 0.688748
Desvio Padrão: 0.008151
Mediana: 0.691683

Métrica MAE
Média: 40.240534
Desvio Padrão: 0.507365
Mediana: 40.074424

Métrica RMSE
Média: 67.912265
Desvio Padrão: 0.892737
Mediana: 67.925669


/kaggle/working/historical_data_occupancy_with_dp_1444.csv

Modelo RF

Métrica R2
Média: 0.694885
Desvio Padrão: 0.006212
Me